In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import json
import time
from pandas import ExcelWriter
import re
import pyodbc
import asyncio
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
from ipywidgets import IntProgress
from IPython.display import display
import os

In [2]:
url_base= 'https://www.lojasrenner.com.br/'

In [3]:
fecha = datetime.date.today()
hoy = fecha.strftime('%Y/%m/%d')

user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'    

chrome_options = Options()  
chrome_options.add_argument("--headless")  
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_options.add_argument("--window-size=1325x744")

browser = webdriver.Chrome('/usr/bin/chromedriver', options = chrome_options)

browser.get(url_base)


In [4]:
#Clickear en FEMENINO
browser.find_element_by_link_text("FEMININO").click()
url_femenino = browser.current_url

#CLICK EN CATEGORIA
boton_categoria = WebDriverWait(browser, 50).until(EC.element_to_be_clickable((By.LINK_TEXT,'CATEGORIA1')))
boton_categoria.click()

#Elementos para generar la lista de categorias
elements = browser.find_elements_by_xpath('//*[@id="item0"]/div[2]/div')

#Lista de categorias
categorias = []
for i in elements:
    categorias.append(i.text.split('\n'))
categorias = categorias[0]


In [5]:
try:
    browser.switch_to.frame("social-push")
    browser.execute_script('document.querySelector("#btClose").click()')
    browser.switch_to.default_content()
except:
    pass
#Se remueven las excepciones
excepts = ['Relógios','Calçados','Bijuterias','Acessórios','Presentinhos','Bolsas','Óculos de Sol','Curve & Plus Size']

[categorias.remove(item) for item in categorias if item in excepts ]


[None, None, None, None, None, None]

In [6]:
categorias

['Bermudas e Shorts',
 'Blusas e Camisetas',
 'Blusões e Suéteres',
 'Calçados',
 'Calças',
 'Camisas',
 'Casacos e Jaquetas',
 'Coletes',
 'Esportivo',
 'Lingerie',
 'Macacões e Jardineiras',
 'Moda Praia',
 'Pijamas',
 'Relógios',
 'Saias',
 'Vestidos']

In [7]:
#CATEGORIAS
categorias_link = []
for item in categorias:
    browser.find_element_by_link_text(item).click()
    categorias_link.append([item,browser.current_url])
    
    browser.get(url_femenino)
    boton_categoria = WebDriverWait(browser, 50).until(EC.element_to_be_clickable((By.LINK_TEXT,'CATEGORIA1')))
    boton_categoria.click()

In [ ]:
#PAGINACIÓN

In [17]:
browser.execute_script("window.scrollTo(0, 5000);")
#browser.save_screenshot('asd.png')

True

In [8]:
print(browser.current_url)

https://www.lojasrenner.com.br/c/feminino/-/N-4zo6za/p1


In [ ]:
browser.current_url

In [10]:
#PAGINACION EN 'cantidad_hojas'
#26/07/ 10.52 a.m
#Clickea en categorias, 
FLAG_COR = False
cantidad_hojas = []
for catlink in categorias_link:
    browser.get(catlink[1])
    
    boton_color = WebDriverWait(browser, 50).until(EC.element_to_be_clickable((By.LINK_TEXT,'COR')))
    boton_color.click()
    elements_colores = browser.find_elements_by_xpath('//*[@id="item3"]/div/div')
    #Lista de colores
    colors = []
    for color in elements_colores:
        colors.append(color.text.split('\n'))
    colors = colors[0]
    for item_color in colors:

        #CLICK EN EL COLOR DE LA LISTA
        time.sleep(4)
        item_color_boton = WebDriverWait(browser, 40).until(EC.element_to_be_clickable((By.LINK_TEXT,item_color)))
        item_color_boton.click()

        #SCROLLEAR A FONDO HASTA QUE CARGUE EL JS LASTPAGINATOR EN EL DOM
        lastPage = -1

        browser.execute_script("window.scrollTo(0, 5000);")

        page = BeautifulSoup(browser.page_source,'html.parser')
        
        if browser.current_url != catlink[1]:
            time.sleep(2)
            lastPage = WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.CLASS_NAME,'js-lastPage')))
            lastPage = int(lastPage.get_attribute('data-last-page'))

        else:
            print(browser.current_url,' ',catlink[1])

        for cant in range(lastPage):
            
            cantidad_hojas.append([catlink[0],item_color,lastPage,browser.current_url])  
            if lastPage > cant+1:
                #SCROLLEAR Y CLICKEAR EN NEXT()
                browser.execute_script("window.scrollTo(0, 5600);")

                btn_next = WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.CLASS_NAME,'js-next-paginator')))
                browser.get(btn_next.get_attribute('href'))
                #btn_next.click()
                FLAG_COR = True
            else:

                if FLAG_COR == True:

                    browser.execute_script("window.scrollTo(0, 100);")
                    filter_row = browser.find_elements_by_xpath('//*[@id="renner-filter"]/div[1]')
                    filters = filter_row[0].find_elements_by_class_name('filter_item')

                    BOTON_COR1 = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.LINK_TEXT,filters[-3].text)))
                    BOTON_COR1.click()  
                    

            #SUBE PARA CLICKEAR EN COLOR
            browser.execute_script("window.scrollTo(0, 100);")
            FLAG_COR = False    
            try:
                time.sleep(2)
                item_color_boton = WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.LINK_TEXT,item_color)))
                item_color_boton.click()
            except:
                b=WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.LINK_TEXT,'COR1')))
                b.click()
            

            browser.execute_script("window.scrollTo(0, 5600);")
            browser.execute_script("window.scrollTo(0, 100);")       


            page = BeautifulSoup(browser.page_source,'html.parser')


TimeoutException: Message: timeout
  (Session info: headless chrome=74.0.3729.131)
  (Driver info: chromedriver=2.41.578700 (2f1ed5f9343c13f73144538f15c00b370eda6706),platform=Linux 4.15.0-66-generic x86_64)


In [17]:
# CATEGORIA, COLOR, NUMERO, LINK
rennerLista = []
for ITEM in cantidad_hojas:
    
    precio = []
    
    browser.get(ITEM[3])
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    page = BeautifulSoup(browser.page_source,'html.parser')
    try:
        articulos = [i.find_all(class_='item_product') for i in page.find_all(class_='wrapper cf results-list js-results-list')][0]
    except:
        #SIGNIFICA QUE NO SE ENCONTRARON RESULTADOS, PAGINA VACIA
        continue
    #element_price = elemento 
    element_price = browser.find_elements_by_class_name('price_product')
    #moverse en el dom para sacar el atributo span class price
    prices = [ene.find_elements_by_class_name('price')[0].text for ene in element_price]

    #SI SPLIT ES 2 SIGNIFICA QUE NO TIENE PRECIO CON DESCUENTO
    #ELSE  SIGNIFICA QUE TIENE PRECIO CON DESCUENTO
    
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    page = BeautifulSoup(browser.page_source,'html.parser')
    try:
        articulos = [i.find_all(class_='item_product') for i in page.find_all(class_='wrapper cf results-list js-results-list')][0]
    except:
        #SIGNIFICA QUE NO SE ENCONTRARON RESULTADOS, PAGINA VACIA
        continue
    
    browser.execute_script("window.scrollTo(0, 2000);")
    time.sleep(0.8)
    browser.execute_script("window.scrollTo(0, 4000);")
    time.sleep(0.8)
    browser.execute_script("window.scrollTo(0, 5000);")#5600
    time.sleep(0.8)
    element_price = browser.find_elements_by_class_name('price_product')
    # class price
    prices = [ene.find_elements_by_class_name('price')[0].text for ene in element_price]
    
    for item in prices:
        if len(item.split()) ==2:
            precio.append([item.split()[1],item.split()[1]])
        else:
            try:
                precio.append([item.split()[1],item.split()[3]])
            except:
                precio.append(['99999','99999'])
            
    indice_aux = 0
    for articulo in articulos:
        data = json.loads(articulo['data-product-gtm'])
        #Imagen del producto
        try:
            img = [i.img['src'] for i in articulo.find_all(class_='img_product')][0]
        except:
            img = [None]
        rennerLista.append([data['id'],ITEM[0],ITEM[1],data['name'],precio[indice_aux][0],precio[indice_aux][1],data['brand'],data['url'],img])
        indice_aux += 1  

In [9]:
#browser.save_screenshot('asd.png')
browser.current_url

'https://www.lojasrenner.com.br/c/feminino/vestidos/-/N-cg003xZ16q1qsyZ1hwylc0/p1'

In [ ]:
*

In [18]:
dfRenner = pd.DataFrame(rennerLista)

In [19]:
#dfRenner.head(1)
dfRenner = dfRenner.rename(columns={0: 'id Producto'})
dfRenner = dfRenner.rename(columns={1: 'Tipo'})
dfRenner = dfRenner.rename(columns={2: 'Color'})
dfRenner = dfRenner.rename(columns={3: 'Descripcion'})
dfRenner = dfRenner.rename(columns={4: 'Precio Original'})
dfRenner = dfRenner.rename(columns={5: 'Precio'})
dfRenner = dfRenner.rename(columns={6: 'Marca'})
dfRenner = dfRenner.rename(columns={7: 'Url Producto'})
dfRenner = dfRenner.rename(columns={8: 'Url Imagen'})

In [20]:
dfRenner['Fecha'] = hoy

In [21]:
dfRenner = dfRenner.drop_duplicates()

In [22]:
dfRenner = dfRenner.reset_index(drop=True)

In [23]:
browser.quit()

In [24]:
#dfRenner["Precio Anterior"] = (dfrenner["Precio Anterior"]
dfRenner['Precio Original'] = (dfRenner['Precio Original'].str.extract(r"([\d,\.]+)", expand=False)
                               .str.replace(",", ".")
                               .astype(float))

In [25]:
dfRenner['Precio'] = (dfRenner['Precio'].str.extract(r"([\d,\.]+)", expand=False)
                               .str.replace(",", ".")
                               .astype(float))

In [26]:
writer = ExcelWriter('./Salida/dfRenner_femenino'+str(fecha)+'.xlsx')
dfRenner.to_excel(writer,'Hoja1')
writer.save()

In [28]:
len(dfRenner)

7115

In [29]:
browser.quit()

5811